In [1]:
import requests


In [2]:
res = requests.get('http://localhost:9200')
print res.content
print res.status_code


{
  "name" : "IHixwm0",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "tXlu5mWFSbSkZgi18vP6lQ",
  "version" : {
    "number" : "5.1.1",
    "build_hash" : "5395e21",
    "build_date" : "2016-12-06T12:36:15.409Z",
    "build_snapshot" : false,
    "lucene_version" : "6.3.0"
  },
  "tagline" : "You Know, for Search"
}

200


In [3]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

In [7]:
import json
r = requests.get('http://localhost:9200')
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+str(i))
    es.index(index='sw',doc_type='people',id=i, body=json.loads(r.content))
    i=i+1
    if i==5: print json.loads(r.content)


{u'starships': [u'http://swapi.co/api/starships/13/'], u'edited': u'2014-12-20T21:17:50.313000Z', u'name': u'Darth Vader', u'created': u'2014-12-10T15:18:20.704000Z', u'url': u'http://swapi.co/api/people/4/', u'gender': u'male', u'vehicles': [], u'skin_color': u'white', u'hair_color': u'none', u'height': u'202', u'eye_color': u'yellow', u'mass': u'136', u'films': [u'http://swapi.co/api/films/6/', u'http://swapi.co/api/films/3/', u'http://swapi.co/api/films/2/', u'http://swapi.co/api/films/1/'], u'species': [u'http://swapi.co/api/species/1/'], u'homeworld': u'http://swapi.co/api/planets/1/', u'birth_year': u'41.9BBY'}


In [8]:
es.get(index='sw', doc_type='people',id=5)

{u'_id': u'5',
 u'_index': u'sw',
 u'_source': {u'birth_year': u'19BBY',
  u'created': u'2014-12-10T15:20:09.791000Z',
  u'edited': u'2014-12-20T21:17:50.315000Z',
  u'eye_color': u'brown',
  u'films': [u'http://swapi.co/api/films/6/',
   u'http://swapi.co/api/films/3/',
   u'http://swapi.co/api/films/2/',
   u'http://swapi.co/api/films/1/',
   u'http://swapi.co/api/films/7/'],
  u'gender': u'female',
  u'hair_color': u'brown',
  u'height': u'150',
  u'homeworld': u'http://swapi.co/api/planets/2/',
  u'mass': u'49',
  u'name': u'Leia Organa',
  u'skin_color': u'light',
  u'species': [u'http://swapi.co/api/species/1/'],
  u'starships': [],
  u'url': u'http://swapi.co/api/people/5/',
  u'vehicles': [u'http://swapi.co/api/vehicles/30/']},
 u'_type': u'people',
 u'_version': 2,
 u'found': True}

In [9]:
r = requests.get('http://localhost:9200')
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+str(i))
    es.index(index='sw',doc_type='people',id=i, body=json.loads(r.content))
    i=i+1
    if i==17: i=i+1
print i

90


In [10]:
es.search(index='sw', body = {"query": {"match": {'name': 'Darth Vader'}}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'4',
    u'_index': u'sw',
    u'_score': 4.082477,
    u'_source': {u'birth_year': u'41.9BBY',
     u'created': u'2014-12-10T15:18:20.704000Z',
     u'edited': u'2014-12-20T21:17:50.313000Z',
     u'eye_color': u'yellow',
     u'films': [u'http://swapi.co/api/films/6/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/2/',
      u'http://swapi.co/api/films/1/'],
     u'gender': u'male',
     u'hair_color': u'none',
     u'height': u'202',
     u'homeworld': u'http://swapi.co/api/planets/1/',
     u'mass': u'136',
     u'name': u'Darth Vader',
     u'skin_color': u'white',
     u'species': [u'http://swapi.co/api/species/1/'],
     u'starships': [u'http://swapi.co/api/starships/13/'],
     u'url': u'http://swapi.co/api/people/4/',
     u'vehicles': []},
    u'_type': u'people'},
   {u'_id': u'44',
    u'_index': u'sw',
    u'_score': 2.0412385,
    u'_source': {u'birth_year

In [13]:
es.search(index='sw', body = {"query": {"prefix": {'name': 'lu'}}})

NotFoundError: TransportError(404, u'index_not_found_exception', u'no such index')

In [12]:
g = open('/home/joey/Dropbox/LAB/category.txt','r')
line = g.readline()
cat_dict = {}
while line != '':
    vec = line.replace('\n','').split('\t')
    if not cat_dict.has_key(vec[0]):
        cat_dict[vec[0]]=vec[1]
    line = g.readline()
g.close()

In [21]:
f = open('/home/joey/Dropbox/Seg_sents.txt','r')
line = f.readline()
i = 1
while line != '':
    if '$cause$' in line:
        vec = line.replace('\n','').split('$')
        content = {}
        tags = []
        tagged_tokens = []
        content['title'] = vec[0]
        content['sent'] = vec[2]
        sen_vec = vec[2].split(' ')
        content['tokens'] = sen_vec
        for item in sen_vec:
            if cat_dict.has_key(item):
                if cat_dict[item] == 'disease' or cat_dict[item] == 'symptom':
                    string = '<disease>'+item+'</disease>'
                    tagged_tokens.append(string)
                    tags.append('disease')
                else:
                    tags.append('NA')
                    tagged_tokens.append('NA')
            else:
                tags.append('NA')
                tagged_tokens.append('NA')
        content['tags'] = tags
        content['tagged_tokens'] = tagged_tokens
        content['id'] = i
        content['index'] = 'baidu'
        es.index(index='baidu',doc_type='cause',id=i, body=content)
        i = i + 1
    line = f.readline()
print i
f.close()

27868


In [17]:
es.search(index='baidu', body = {"query": {"match": {'title': '外阴假上皮瘤样增生'}}}) 

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'baidu',
    u'_score': 30.375046,
    u'_source': {u'id': 1,
     u'index': u'baidu',
     u'sent': u'\u5177\u4f53 \u75c5\u56e0 \u4e0d\u8be6 \uff0c \u53ef\u80fd \u4e0e \u5916\u9634 \u6f6e\u6e7f \u6216 \u5206\u6ccc\u7269 \u957f\u671f \u523a\u6fc0 \u6709\u5173 \u3002',
     u'tagged_tokens': [u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA'],
     u'tags': [u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA'],
     u'title': u'\u5916\u9634\u5047\u4e0a\u76ae\u7624\u6837\u589e\u751f',
     u'tokens': [u'\u5177\u4f53',
      u'\u75c5\u56e0',
      u'\u4e0d\u8be6',
      u'\uff0c',
      u'\u53ef\u80fd',
      u'\u4e0e',
      u'\u5916\

In [18]:
es.search(index='baidu', body = {"query": {"bool": {"must": [{"match": {'title': '小儿支气管扩张'}},{"match": {'tokens': '体液免疫缺陷'}}]}}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'11',
    u'_index': u'baidu',
    u'_score': 50.31079,
    u'_source': {u'id': 11,
     u'index': u'baidu',
     u'sent': u'\u4e3b\u8981 \u5305\u62ec \u4f53\u6db2\u514d\u75ab\u7f3a\u9677 \u3001 \u5c40\u90e8 \u514d\u75ab \u9632\u5fa1 \u7f3a\u9677 \u548c \u514d\u75ab \u7d0a\u4e71 \u3002',
     u'tagged_tokens': [u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA'],
     u'tags': [u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA',
      u'NA'],
     u'title': u'\u5c0f\u513f\u652f\u6c14\u7ba1\u6269\u5f20',
     u'tokens': [u'\u4e3b\u8981',
      u'\u5305\u62ec',
      u'\u4f53\u6db2\u514d\u75ab\u7f3a\u9677',
      u'\u3001',
      u'\u5c40\u90e8',
      u'\u514d\u75ab',
      u'\u9632\u5fa1',
      u'\u7f3a\u9677',
    

In [19]:
body = {
        "query": {
            "bool": {
                "must": [
                    {"match": {"title": '外阴假上皮瘤样增生'}},
                    {"match": {"tagged_tokens": '人类乳头瘤病毒'}}
                ]
            }
        }
    }

es.count(index='baidu',body=body)['count']

226

In [20]:
print u'\u5176\u4e2d \uff0c \u6700 \u591a\u89c1\u4e8e \u4f53\u6db2\u514d\u75ab\u7f3a\u9677 \u7684 \u75c5\u4eba \uff0c \u5982 X \u8fde\u9501 \u7684 \u4f4e\u4e19\u79cd\u7403\u86cb\u767d\u8840\u75c7 \u3001 \u666e\u901a \u53d8\u5f02\u578b\u514d\u75ab\u7f3a\u9677\u75c5 \uff0c IgG \u4e9a\u7c7b \u7f3a\u9677 \u4e5f \u4e3a \u652f\u6c14\u7ba1\u6269\u5f20 \u7684 \u75c5\u56e0 \u4e4b\u4e00 \u3002'

其中 ， 最 多见于 体液免疫缺陷 的 病人 ， 如 X 连锁 的 低丙种球蛋白血症 、 普通 变异型免疫缺陷病 ， IgG 亚类 缺陷 也 为 支气管扩张 的 病因 之一 。
